### Relations:

In Django, relationships between models play a crucial role in structuring and organizing data. These relationships define how different entities are connected to each other in a database, enabling powerful data querying and manipulation capabilities.

There are several types of relationships supported by Django, including:

    One-to-Many (ForeignKey): In this type of relationship, each record in one model can be associated with multiple records in another model. This is achieved by using a ForeignKey field in the model that "owns" the relationship.

    Many-to-One (Reverse ForeignKey): This is the reverse of a ForeignKey relationship. Each record in one model can be associated with only one record in another model, but the second model can have multiple records related to it.

    Many-to-Many (ManyToManyField): This type of relationship allows each record in one model to be associated with multiple records in another model, and vice versa. This is useful for representing complex relationships where multiple entities can be connected to each other in various combinations.

Understanding and effectively utilizing these relationships is essential for designing robust database schemas and building efficient Django applications. By defining and managing relationships between models, you can create sophisticated data structures that accurately represent real-world scenarios and support complex data operations.

Let's explain each of these relationship types with examples:
One-to-Many (ForeignKey):

In a One-to-Many relationship, also known as a ForeignKey relationship, each record in one model can be associated with multiple records in another model. However, each record in the second model is associated with only one record in the first model. This relationship is established by using a ForeignKey field in the model that "owns" the relationship.

Example:
Consider a scenario where we have two models: Author and Book. Each author can write multiple books, but each book is written by only one author. 
In this example, the Book model has a ForeignKey field author that references the Author model. This establishes a One-to-Many relationship, where each book is associated with one author, but each author can have multiple books.

Many-to-One (Reverse ForeignKey):

A Many-to-One relationship is the reverse of a ForeignKey relationship. In this type of relationship, each record in one model can be associated with only one record in another model, but the second model can have multiple records related to it. This is achieved by using a ForeignKey field in the second model.

Example:
Building upon the previous example, let's consider the Many-to-One relationship from the perspective of the Author model. Each author can have multiple books, but each book is authored by only one author. 
In this example, the Book model includes a ForeignKey field author that references the Author model. Each book is associated with one author, creating a Many-to-One relationship between books and authors.

Many-to-Many (ManyToManyField):

In a Many-to-Many relationship, each record in one model can be associated with multiple records in another model, and vice versa. This type of relationship is useful for representing complex relationships where multiple entities can be connected to each other in various combinations. This is achieved by using a ManyToManyField in both models.

Example:
Consider a scenario where we have two models: Student and Course. Each student can enroll in multiple courses, and each course can have multiple students enrolled.

In this example, both the Student and Course models include a ManyToManyField. This establishes a Many-to-Many relationship, where each student can be associated with multiple courses, and each course can have multiple students enrolled.

Lets add a new data model to our models.py file.

This models is supposed to hold data regarding actors.

1- Navigate to your app's models.py file
2- Create a new class called Actor
3- This class have 2 attributes: first_name, and last_name. 
4- The first_name and last_name attribute are Charfields that can not be left null, and have the max length of 100 characters (you may want to label them as an db_index so you perform faster searching on it in the future)<br>
5- Now, you can change the field type of the main_act attribute in the Movie class. Change it to model.ForeignKey.
6- The ForeignKey field type requires two arguments. 
- The first one is the name of the class that you want to link to this field (Movie).
- And the second one is the on_delete argument. You may ask why?... well the first thing that comes to mind is that what if an actors data get deleted from the first class! what should happen to the row that was connected to this actor? and you have various options here:
    - You can tell Django that if an actor data is deleted, the row associated to that actor should also get deleted. This is called 'CASCADE' and if you want it, you should pass models.CASCADE as the second argument to the ForeignKey field.
    - You can also tell Django that if an actor data is being used in here, then is should not be possible to delete that actor from the table. This is called 'PROTECT' and if you want it you should pass models.PROTECT as the second argument to the ForeignKey field.
    - Another option that you have here is 'SET_NULL' which basically tells django to set this field to null if the associated actor gets deleted from the first table (models.SET_NULL).
    - There are other options too that you can check in Django Documentation.
7- Set the ForeignKey's second argument to CASCADE

Your models.py file should look like the following:

In [ ]:
from django.db import models
from django.core.validators import MinValueValidator, MaxValueValidator
from django.urls import reverse
from django.utils.text import slugify


class Actor(models.Model):
    first_name = models.CharField(max_length=100, null=False)
    last_name  = models.CharField(max_length=100, null=False)


class Movie(models.Model):
    title = models.CharField(max_length=50)
    rating = models.IntegerField(validators=[MinValueValidator(1), MaxValueValidator(10)])
    main_act= models.ForeignKey(Actor, models.CASCADE)
    is_bestselling = models.BooleanField(default=False)
    slug = models.SlugField(default='', null=False, db_index=True)
 
    def get_absolute_url(self):
        return reverse("movie_detail_url", args=[self.slug])

    def __str__(self):
        return f"title:{self.title}, rating:{self.rating}, Main Actor/actress:{self.main_act}{', Best Seller' if self.is_bestselling else ''}"

Now, you can save everything, stop the development server, and makemigrations command.<br>

You will get an error which you have seen before:

In [ ]:
It is impossible to change a nullable field 'main_act' on movie to non-nullable without providing a default. This is because the database needs something to populate existing rows.
Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows with a null value for this column)
 2) Ignore for now. Existing rows that contain NULL values will have to be handled manually, for example with a RunPython or RunSQL operation.
 3) Quit and manually define a default value in models.py.

Do you know why got this error? let me explain:

The error is telling you that it is impossible to change a nullable field to a non_nullable field without providing a default. Still don't get it?
Ok let me explain more: The main_Act attribute (field) used to be a CharField, and now we want to change it to a brand new field (ForiednKey), this means that this field will no longer hold strings and instead holds links (addresses) to other fields.

And do we have any entries in the Actor table? even if we had. To which row of the table should we point?<br>
This is the reason that django is yelling at you!

1- Use the 3rd option to stop the terminal.<br>
2- Get back to models.py and give the main_act field the option to be null (null=True).<br>
3- Run the makemigrations command and then migrate to finialize the changes.<br>

Now, you face a new problem!<br>
Let me explain why Django is again yelling at you.<br>

The Actor table (model) is something that we have just made! which means that it is empty. However there are a number of rows in the Movie table, and those rows have values for their main_act field.<br>
Those values do not match any rows in the Actor table, and that is why django is yelling at you.<br>

Well, unfortunately there is no fix to this!<br>
You either need to enter your shell and manually set null for all the mina_act fields in the Movie table, or just simply delete the whole data inside that table so you can later populate it with new values.<br>

lets do the second one:<br>
1- Enter the shell, and use the following command to delete all the existing data inside the Movie table.

In [ ]:
Movie.objects.all().delete()

2- Now, you can run the migrate command, and it succides.

3- To make it easier to work with the new data model, go to your admin.py and make sure that you can access the new data model from the adminstrative section.

In [ ]:
class ActorAdmin(admin.ModelAdmin):
    list_display = ['first_name', 'last_name']

Don't forget to register this new class otherwise you won't be able to see it in the admin page.

In [ ]:
admin.site.register(Actor, ActorAdmin)

Your admin.py file should look like the following block:

In [ ]:
from django.contrib import admin
from .models import Movie, Actor

class ActorAdmin(admin.ModelAdmin):
    list_display = ['first_name', 'last_name']


class MovieAdmin(admin.ModelAdmin):
    prepopulated_fields = {'slug':('title',)}
    list_filter = ('rating', 'is_bestselling')
    list_display = ['id', 'title', 'rating', 'is_bestselling', 'slug']

admin.site.register(Movie, MovieAdmin)
admin.site.register(Actor, ActorAdmin)

In the next notebook we will explore new python commands that we can perform on our data models.